In [ ]:
# Load dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine, text
import pymc as pm
import arviz as az
from scipy.stats import zscore

# Import parsing functions
import sys
sys.path.append('/Users/JO/PhD/neurocritical-transfers/utils')
import hems
import metar_parse

# Define the SQLalchemy engine
engine = create_engine(f"sqlite:////Users/JO/PhD/neurocritical-transfers/data/db.sqlite")

# Read the SQL query from the file
with open('/Users/JO/PhD/neurocritical-transfers/notes/final-analysis/0-database-query/primary-based-transfer-query.sql', 'r') as file:
    query = file.read()

# Execute the query
d = pd.read_sql(query, engine)

In [ ]:
# Define paths to data
PATH_METAR='/Users/JO/PhD/neurocritical-transfers/data/metar'
PATH_AIRPORT_COORDS='/Users/JO/PhD/neurocritical-transfers/data/icu-airport-mapping-with-coordinates.csv'

# Load and parse weather data
weather_parsed = metar_parse.get_metar_data(PATH_METAR, PATH_AIRPORT_COORDS)
weather = weather_parsed.copy()
weather['hems_minima'] = weather.apply(hems.hems_minima, axis=1)
weather = hems.hems_minima_window(weather, window='121Min')

# Get the list of unique airports
airports = weather['icao'].unique()

# Add month to dataset
weather['month'] = weather['time_utc'].dt.month

# Calculate the number of rows and columns for subplots
num_airports = len(airports)
num_cols = 3  # Number of columns
num_rows = -(-num_airports // num_cols)  # Ceiling division to get the number of rows

# national mean 
countrywide_mean = weather.query("icao not in ['ESGT', 'ESGP']").groupby('month')['hems_minima'].mean() * 100

In [ ]:
# Remmap receiving hospital codes to names
hospitals_map = {"11001": "Karolinska universitetssjukhuset, Solna",
 "11003": "Karolinska universitetssjukhuset, Solna",
 "51001": "Sahlgrenska universitetssjukhuset",
 "12001": "Akademiska sjukhuset",
 "21001": "Universitetssjukhuset i Linköping",
 "64001": "Norrlands universitetssjukhus",
 "41001": "Universitetssjukhuset i Lund",
 "41002": "Universitetssjukhuset i Lund"}
d['tertiary_center'] = d.tertiary_center_id.map(hospitals_map)

# Fix admission time so that time zone is clear, also add a admission time in UTC for later merging of data
d["sir_adm_time"] = pd.to_datetime(d["sir_adm_time"], unit='s').dt.tz_localize('Europe/Stockholm')
d['admission_time_utc'] = d["sir_adm_time"].dt.tz_convert('UTC')

# Load dataframes of formatted ICU names and ICU to airport mappings
icu_names = pd.read_csv('/Users/JO/PhD/neurocritical-transfers/data/icu-mapping-names-only.csv', sep=";")
airports = pd.read_csv('/Users/JO/PhD/neurocritical-transfers/data/icu-airport-mapping.csv', sep=";")
airports_sending = airports.rename({'formatted_icu_name': 'sir_icu_name'}, axis=1).add_prefix('sending_')
airports_receiving = airports.rename({'formatted_icu_name': 'tertiary_center'}, axis=1).add_prefix('receiving_')

# Remap ICU names to properly formatted ICU names
icu_names_dict = dict(zip(icu_names['sir_icu_name'], icu_names['formatted_icu_name']))
d.sir_icu_name = d.sir_icu_name.map(icu_names_dict)

# Merge dataframes with sending and receiving airports dataframes
d_with_sending_icao = pd.merge(d, airports_sending, left_on='sir_icu_name', right_on='sending_sir_icu_name',  how='left')
d_with_receiving_icao = pd.merge(d_with_sending_icao, airports_receiving, left_on='tertiary_center', right_on='receiving_tertiary_center',  how='left')
d_with_airports = d_with_receiving_icao.sort_values(by=['admission_time_utc'])

# "Duplicate" the weather dataframe into two separate df with prefixes "sending" and "receiving" before merging with the clinical data
sending_weather = weather.sort_values(by=['time_utc']).add_prefix('sending_')
receiving_weather = weather.sort_values(by=['time_utc']).add_prefix('receiving_')

# Merge the dataframe with METAR observations and HEMS inferences for both sending and receving hospitals within 30 minutes of the ICU admission
s_df = pd.merge_asof(d_with_airports, sending_weather, by='sending_icao', left_on='admission_time_utc', right_on='sending_time_utc', tolerance=pd.Timedelta(30, unit='min'), direction='nearest')
r_df = pd.merge_asof(s_df, receiving_weather, by='receiving_icao', left_on='admission_time_utc', right_on='receiving_time_utc', tolerance=pd.Timedelta(30, unit='min'), direction='nearest')

# Check is HEMS minima at point estimates or windows are met at both places
r_df['both_point_hems_minima'] = (
    (r_df['sending_hems_minima'] & r_df['receiving_hems_minima'])
    .where(
        ~r_df['sending_hems_minima'].isna() & ~r_df['receiving_hems_minima'].isna(), np.nan)
)

r_df['both_hems_minima_window'] = (
    (r_df['sending_hems_minima_window'] & r_df['receiving_hems_minima_window'])
    .where(
        ~r_df['sending_hems_minima_window'].isna() & ~r_df['receiving_hems_minima_window'].isna(), np.nan)
)

# Choose columns
final_d = r_df

# Limit the data to hospitals that are presumed to be frequent users of hems
hems_users = ["Arvika IVA", "Bollnäs IVA", "Eskilstuna IVA", "Falun IVA", "Gällivare IVA", "Gävle IVA", "Hudiksvall IVA", "Karlstad IVA", "Lycksele IVA", "Mora IVA",  "Östersund IVA", "Skövde IVA", "Torsby IVA", "Trollhättan IVA", "Västerås IVA", "Visby IVA"]

hems_d = final_d[final_d.sir_icu_name.isin(hems_users)]
hems_d['utc_month']=hems_d.admission_time_utc.dt.month
hems_d['utc_day']=hems_d.admission_time_utc.dt.dayofyear
hems_d['utc_hour']=hems_d.admission_time_utc.dt.hour

In [ ]:
hems_d.to_csv('hems_d.csv')